In [2]:
#DS2002 Project 2
#Kendle Schooler & Adair Hancock

import json
import pandas as pd
import requests
import sqlite3

In [4]:
#JSON input:
url='https://api.carbonintensity.org.uk/intensity/factors'
headers = {
  'Accept': 'application/json'}
#API call is made.
response = requests.get(url, params={}, headers = headers)

if response.status_code==200:
  data = response.json()
  #Response is converted to Pandas dataframe.
  df = (pd.json_normalize(data['data']))
  rows=len(df)
  cols=len(df.columns)
  df['Imports']=df['Dutch Imports']+df['French Imports']+df['Irish Imports']
  df.drop(['Dutch Imports','French Imports','Irish Imports'],axis=1,inplace=True)
  df['Gas']=df['Gas (Open Cycle)']+df['Gas (Combined Cycle)']
  df.drop(['Gas (Open Cycle)','Gas (Combined Cycle)'],axis=1,inplace=True)
  #User chooses final output type.
  format=input("Choose an output format(csv,sql):")
  if format=='csv':
    #Converts dataframe to csv.
    csv=df.to_csv('ci_data.csv',index=False)
  elif format=='sql':
    #Connects to SQL database.
    conn = sqlite3.connect('carbon.db')
    #Creates 'CarbonIntensity' table from dataframe and writes to 'carbon' SQL database.
    df.to_sql('CarbonIntensity', conn, if_exists='replace', index=False)
    conn.close
  else:
    print('Unable to convert to this format. Please choose another.')
elif response.status_code==400:
  print('Error: Bad request')
elif response.status_code==500:
  print('An internal server error has occurred')
else:
  print('Sorry, an unknown error occurred.')

print('Summary:')
print('Starting num. columns=',cols)
print("Starting num. records=",rows)
print('Ending num. columns=',len(df.columns))
print('Ending num. records=',len(df))

   Biomass  Coal  Dutch Imports  French Imports  Gas (Combined Cycle)  \
0      120   937            474              53                   394   

   Gas (Open Cycle)  Hydro  Irish Imports  Nuclear  Oil  Other  \
0               651      0            458        0  935    300   

   Pumped Storage  Solar  Wind  
0               0      0     0  
   Biomass  Coal  Hydro  Nuclear  Oil  Other  Pumped Storage  Solar  Wind  \
0      120   937      0        0  935    300               0      0     0   

   Imports   Gas  
0      985  1045  
Choose an output format(csv,sql):csv
Summary:
Starting num. columns= 14
Starting num. records= 1
Ending num. columns= 11
Ending num. records= 1


In [ ]:
#CSV input:
#Before starting, ensure the correct csv file has been mounted.

#CSV is converted to Pandas dataframe.
data2 = pd.read_csv('most-polluted-countries.csv')
csv2=data2.drop(['ccn3','pollution_density_per_Mile','share_borders','mostPollutedCountries_particlePollution'], axis=1)
#User chooses final output type.
output = input("Choose an output format (json, sql):")
if output=='sql':
  #Connects to SQL database.
  conn = sqlite3.connect('pollution.db')
  #Creates 'Countries' table from dataframe and writes to 'pollution' SQL database.
  csv2.to_sql('Countries', conn, if_exists='replace', index=False)
  conn.close
elif output=='json':
  #Converts dataframe to json string then to json dictionary.
  mpc_json = json.loads(csv2.to_json(orient="records"))
else:
  print('Unable to convert to this format. Please choose another.')

print('Summary:')
print('Starting num. columns=',len(data2.columns))
print("Starting num. records=",len(data2))
print('Ending num. columns=',len(csv2.columns))
print('Ending num. records=',len(csv2))


Choose an output format (json, sql):json
Summary:
Starting num. columns= 12
Starting num. records= 96
Ending num. columns= 8
Ending num. records= 96
